<a href="https://colab.research.google.com/github/MjunMJ/python_playground/blob/main/converting_different_encodings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import base64
import binascii

#standard prefixes - 0b for bin(), 0o for oct() and 0x for hex()

In [ ]:
hex_value = "4765656b73666f724765656b73"
string = 'GeeksforGeeks'

#hex to base 16
base = 16
result = int(hex_value, base)
print(f"int: {result}")
print(format(0x4765656b73666f724765656b73, 'd')) # alternative way to get int

#hex to bytes
bytes_value = bytes.fromhex(hex_value)
print(f"bytes: {bytes_value}")

#hex to base64
base64_value = base64.b64encode(bytes_value)
print(f"base64: {base64_value}")

#bytes to ascii or utf-8
decoded = bytes_value.decode('ascii')
print(f"decoded: {decoded}")

#string to bytes (same as getting ascii)
string_to_bytes = bytes(string, 'utf-8')
print(f"string to bytes: {string_to_bytes}") # OR string.encode('ascii')

#bytes to bits
bits = bin(int.from_bytes(bytes_value, byteorder='big'))[2:]
print(f"bits; {bits}")

#bits back to string, via bytes
bytes_from_bits = int(bits, 2).to_bytes((len(bits) + 7) // 8, byteorder='big')
print(f"bytes from bits: {bytes_from_bits}")

string = bytes_from_bits.decode('ascii')
print(f"string: {string}")

#string to hex
translated_hex = string.encode('ascii').hex()
print(f"encoded hex_value: {translated_hex}")

#hex to bin
binary = bin(result)[2:]  # [2:] removes the '0b' prefix
print(binary)


int: 5656580133434475378123357907827
5656580133434475378123357907827
bytes: b'GeeksforGeeks'
base64: b'R2Vla3Nmb3JHZWVrcw=='
decoded: GeeksforGeeks
string to bytes: b'GeeksforGeeks'
bits; 1000111011001010110010101101011011100110110011001101111011100100100011101100101011001010110101101110011
bytes from bits: b'GeeksforGeeks'
string: GeeksforGeeks
encoded hex_value: 4765656b73666f724765656b73
1000111011001010110010101101011011100110110011001101111011100100100011101100101011001010110101101110011


In [52]:
hex_qq ="1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"

bytes_ciphertext = bytes.fromhex(hex_qq)

print(bytes_ciphertext)
print(len(bytes_ciphertext))

b'\x1b77316?x\x15\x1b\x7f+x413=x9x(7-6<x7>x:9;76'
34


In [2]:
#conduct XOR
def xor(key, ciphertext):
  key_byte = bytes([key])
  keystream = key_byte*len(ciphertext)
  return ([x^y for (x,y) in zip(ciphertext, keystream)])

#loop through all 256 iterations
dict_all_outputs = {}

for i in range(256):
  decrypted = xor(i, bytes_ciphertext)
  len_decrypted = len(decrypted)
  count_is_alpha=0
  for j in decrypted:
    if (j >=65) and (j<=122):
      count_is_alpha+=1
    else:continue
  score = count_is_alpha/len_decrypted
  dict_all_outputs[i] = score


print(sorted(dict_all_outputs.items(), key=lambda x: x[1], reverse=True))



[(88, 0.7941176470588235), (89, 0.7941176470588235), (90, 0.7941176470588235), (92, 0.7941176470588235), (93, 0.7941176470588235), (94, 0.7941176470588235), (95, 0.7941176470588235), (97, 0.7941176470588235), (98, 0.7941176470588235), (99, 0.7941176470588235), (108, 0.7941176470588235), (111, 0.7941176470588235), (112, 0.7941176470588235), (114, 0.7941176470588235), (117, 0.7941176470588235), (120, 0.7941176470588235), (81, 0.7647058823529411), (82, 0.7647058823529411), (105, 0.7647058823529411), (106, 0.7647058823529411), (109, 0.7647058823529411), (110, 0.7647058823529411), (116, 0.7647058823529411), (122, 0.7647058823529411), (123, 0.7647058823529411), (124, 0.7647058823529411), (125, 0.7647058823529411), (126, 0.7647058823529411), (127, 0.7647058823529411), (80, 0.7352941176470589), (83, 0.7352941176470589), (84, 0.7352941176470589), (87, 0.7352941176470589), (91, 0.7352941176470589), (96, 0.7352941176470589), (100, 0.7352941176470589), (101, 0.7352941176470589), (102, 0.7352941176

In [3]:
!pip install nltk

In [4]:
import nltk
from nltk.corpus import wordnet

nltk.download('wordnet')  # Download WordNet data (only needed once)

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [53]:

dict_above_7 = {key: value for key, value in dict_all_outputs.items() if value >= 0.7}

def check_if_readable_text(text):
    synset = wordnet.synsets(text)
    if len(synset) == 0:
      return False
    else:
      return True

#print(check_if_readable_text(bytes([88])))
#print(check_if_readable_text(bytes("lol","ascii").decode("ascii")))

hex_qq ="1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"

bytes_ciphertext = bytes.fromhex(hex_qq)

key_percentTrue = {}

for key, value in dict_above_7.items():
  letter = bytes([key]).decode("ascii")
  phrase = bytes(xor(key,bytes_ciphertext)).decode("ascii").split()
  if len(phrase)==1:continue
  else:
    list_readability = [check_if_readable_text(w) for w in phrase]
    percent_True = list_readability.count(True)/len(phrase)
    key_percentTrue[key] = percent_True

max_percentTrue =max(key_percentTrue.values())

answer = [key for key in key_percentTrue if key_percentTrue[key] == max_percentTrue][0]

print(bytes([answer]).decode("ascii"))
print(bytes(xor(answer,bytes_ciphertext)).decode("ascii"))

X
Cooking MC's like a pound of bacon
